Задача 1.

**Гетерогенная параллелизация** — это подход к параллельным вычислениям, при котором в рамках одной программы или системы совместно используются **разные типы вычислительных устройств**, чаще всего **CPU и GPU**, а иногда также FPGA, TPU и другие ускорители.

---

### 1. Различия между параллельными вычислениями на CPU и GPU

**CPU (Central Processing Unit):**

* Имеет небольшое количество мощных ядер (обычно от 4 до 32).
* Оптимизирован для последовательных и слабо параллельных задач.
* Обладает сложной логикой управления, большим кэшем, эффективен при ветвлениях и условных операциях.
* Используется для управления программой, выполнения логики, ввода-вывода.

**GPU (Graphics Processing Unit):**

* Содержит тысячи более простых вычислительных ядер.
* Оптимизирован для массового параллелизма и однотипных операций над большими массивами данных.
* Наиболее эффективен для задач с высокой степенью параллелизма и минимальным ветвлением.
* Используется для вычислений над векторами, матрицами, изображениями, нейронными сетями.

CPU лучше подходит для сложной логики и управления, GPU — для интенсивных численных вычислений с высокой степенью параллелизма.

---

### 2. Преимущества гетерогенной параллелизации

* **Повышение производительности** за счёт распределения задач между CPU и GPU.
* **Энергоэффективность**: GPU выполняет массовые вычисления быстрее и с меньшими затратами энергии.
* **Гибкость**: разные части программы могут быть оптимизированы под разные архитектуры.
* **Масштабируемость**: возможность подключения дополнительных ускорителей.
* **Оптимальное использование ресурсов** системы.

---

### 3. Примеры реальных приложений гетерогенной параллелизации

* **Машинное обучение и искусственный интеллект**
  Обучение нейронных сетей (TensorFlow, PyTorch): CPU управляет процессом, GPU выполняет матричные операции.

* **Обработка изображений и видео**
  Кодирование видео, фильтрация изображений, компьютерное зрение (OpenCV с CUDA).

* **Научные и инженерные расчёты**
  Моделирование климата, молекулярная динамика, численные методы в физике и химии.

* **Игровые движки**
  CPU отвечает за логику игры и физику, GPU — за рендеринг графики и шейдеры.

* **Финансовые вычисления**
  Анализ больших массивов данных, моделирование рисков, высокочастотный трейдинг.

---

### В итоге

Гетерогенная параллелизация позволяет эффективно сочетать сильные стороны CPU и GPU, обеспечивая высокую производительность и оптимальное использование вычислительных ресурсов. Этот подход является ключевым в современных высокопроизводительных и ресурсоёмких приложениях.


In [1]:
%%writefile task2.cpp

#include <iostream>
#include <vector>
#include <cstdlib>
#include <ctime>
#include <omp.h>

int main() {
    const int N = 10000;              // Размер массива
    std::vector<int> arr(N);          // Динамический массив на N элементов

    // Инициализация генератора случайных чисел
    srand(time(0));

    // Заполняем массив случайными числами
    for (int i = 0; i < N; i++)
        arr[i] = rand();

    // ------------------- Последовательная версия -------------------

    // Засекаем время начала
    double start = omp_get_wtime();

    // Изначально считаем, что минимум и максимум — первый элемент
    int minVal = arr[0], maxVal = arr[0];

    // Обычный проход по массиву
    for (int i = 1; i < N; i++) {
        if (arr[i] < minVal) minVal = arr[i];   // Обновляем минимум
        if (arr[i] > maxVal) maxVal = arr[i];   // Обновляем максимум
    }

    // Засекаем время окончания
    double end = omp_get_wtime();

    // Вывод результата и времени работы
    std::cout << "Sequential min: " << minVal
              << " max: " << maxVal
              << " time: " << end - start << std::endl;

    // ------------------- Параллельная версия -------------------

    // Снова засекаем время
    start = omp_get_wtime();

    // Сбрасываем минимум и максимум
    minVal = arr[0];
    maxVal = arr[0];

    // Параллельный цикл
    // В конце OpenMP объединяет результаты
    #pragma omp parallel for reduction(min:minVal) reduction(max:maxVal)
    for (int i = 0; i < N; i++) {
        if (arr[i] < minVal) minVal = arr[i];
        if (arr[i] > maxVal) maxVal = arr[i];
    }

    // Конец замера времени
    end = omp_get_wtime();

    // Вывод результата и времени работы параллельной версии
    std::cout << "Parallel min: " << minVal
              << " max: " << maxVal
              << " time: " << end - start << std::endl;

    return 0;
}



Writing task2.cpp


In [2]:
! g++ -fopenmp -o2 task2.cpp -o anyname
!./anyname

Sequential min: 17029 max: 2147205612 time: 7.033e-05
Parallel min: 17029 max: 2147205612 time: 0.00184204


In [3]:
%%writefile task3.cpp
#include <iostream>
#include <vector>
#include <cstdlib>
#include <ctime>
#include <omp.h>
#include <algorithm>

// Заполнение массива случайными числами
void fillArray(std::vector<int>& a) {
    for (int& x : a)
        x = rand() % 100000;
}

// Последовательная сортировка выбором
void selectionSortSequential(std::vector<int>& a) {
    int n = a.size();

    for (int i = 0; i < n - 1; i++) {
        int minIndex = i;

        // Поиск минимального элемента
        for (int j = i + 1; j < n; j++) {
            if (a[j] < a[minIndex])
                minIndex = j;
        }

        // Перестановка элементов
        std::swap(a[i], a[minIndex]);
    }
}

// Параллельная сортировка выбором с OpenMP
void selectionSortParallel(std::vector<int>& a) {
    int n = a.size();

    for (int i = 0; i < n - 1; i++) {
        int minIndex = i;

        // Параллельный поиск минимального элемента
        #pragma omp parallel for
        for (int j = i + 1; j < n; j++) {

            // Защита общей переменной minIndex
            #pragma omp critical
            {
                if (a[j] < a[minIndex])
                    minIndex = j;
            }
        }

        // Обмен найденного минимума
        std::swap(a[i], a[minIndex]);
    }
}

int main() {
    srand(time(nullptr));

    const int N1 = 1000;
    const int N2 = 10000;

    // ====== Тест для массива из 1000 элементов ======
    std::vector<int> arr1(N1);
    fillArray(arr1);

    std::vector<int> arr1Copy = arr1;

    double start = omp_get_wtime();
    selectionSortSequential(arr1);
    double end = omp_get_wtime();
    std::cout << "Sequential sort (1000): " << end - start << " sec\n";

    start = omp_get_wtime();
    selectionSortParallel(arr1Copy);
    end = omp_get_wtime();
    std::cout << "Parallel sort (1000): " << end - start << " sec\n\n";

    // ====== Тест для массива из 10000 элементов ======
    std::vector<int> arr2(N2);
    fillArray(arr2);

    std::vector<int> arr2Copy = arr2;

    start = omp_get_wtime();
    selectionSortSequential(arr2);
    end = omp_get_wtime();
    std::cout << "Sequential sort (10000): " << end - start << " sec\n";

    start = omp_get_wtime();
    selectionSortParallel(arr2Copy);
    end = omp_get_wtime();
    std::cout << "Parallel sort (10000): " << end - start << " sec\n";

    return 0;
}


Writing task3.cpp


In [4]:
! g++ -fopenmp -o2 task3.cpp -o anyname
!./anyname

Sequential sort (1000): 0.00210048 sec
Parallel sort (1000): 0.0149562 sec

Sequential sort (10000): 0.319791 sec
Parallel sort (10000): 2.9411 sec


In [5]:
%%writefile task4.cpp
#include <iostream>
#include <vector>
#include <cstdlib>
#include <ctime>
#include <cuda_runtime.h>

// Заполнение массива случайными числами
void fillArray(std::vector<int>& a) {
    for (int& x : a)
        x = rand() % 100000;
}
__global__ void mergeKernel(int* input, int* output, int width, int n) {

    // Глобальный индекс потока
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Левая граница подмассива
    int left = idx * 2 * width;

    if (left < n) {
        int mid = min(left + width, n);        // середина
        int right = min(left + 2 * width, n);  // правая граница

        int i = left;
        int j = mid;
        int k = left;

        // Слияние двух отсортированных подмассивов
        while (i < mid && j < right) {
            if (input[i] <= input[j])
                output[k++] = input[i++];
            else
                output[k++] = input[j++];
        }

        // Копирование оставшихся элементов
        while (i < mid)
            output[k++] = input[i++];
        while (j < right)
            output[k++] = input[j++];
    }
}

void mergeSortGPU(std::vector<int>& data) {
    int n = data.size();
    int* d_input;
    int* d_output;

    size_t size = n * sizeof(int);

    // Выделение памяти на GPU
    cudaMalloc(&d_input, size);
    cudaMalloc(&d_output, size);

    // Копирование данных на GPU
    cudaMemcpy(d_input, data.data(), size, cudaMemcpyHostToDevice);

    int threadsPerBlock = 256;
    int blocks;

    // Итеративное увеличение размера подмассивов
    for (int width = 1; width < n; width *= 2) {
        blocks = (n + (2 * width * threadsPerBlock) - 1) / (2 * width * threadsPerBlock);

        mergeKernel<<<blocks, threadsPerBlock>>>(d_input, d_output, width, n);

        // Обмен указателей
        std::swap(d_input, d_output);
    }

    // Копирование результата обратно на CPU
    cudaMemcpy(data.data(), d_input, size, cudaMemcpyDeviceToHost);

    // Освобождение памяти
    cudaFree(d_input);
    cudaFree(d_output);
}

int main() {
    srand(time(nullptr));

    const int N1 = 10000;
    const int N2 = 100000;

    std::vector<int> arr1(N1);
    fillArray(arr1);

    std::vector<int> arr2(N2);
    fillArray(arr2);

    // ===== Массив 10 000 =====
    clock_t start = clock();
    mergeSortGPU(arr1);
    clock_t end = clock();

    std::cout << "GPU merge sort (10000): "
              << double(end - start) / CLOCKS_PER_SEC
              << " sec\n";

    // ===== Массив 100 000 =====
    start = clock();
    mergeSortGPU(arr2);
    end = clock();

    std::cout << "GPU merge sort (100000): "
              << double(end - start) / CLOCKS_PER_SEC
              << " sec\n";

    return 0;
}


Writing task4.cpp


In [6]:
! g++ -fopenmp -o2 task4.cpp -o anyname
!./anyname

task4.cpp:5:10: fatal error: cuda_runtime.h: No such file or directory
    5 | #include <cuda_runtime.h>
      |          ^~~~~~~~~~~~~~~~
compilation terminated.
Sequential sort (1000): 0.00224574 sec
Parallel sort (1000): 0.0168591 sec

Sequential sort (10000): 0.208004 sec
Parallel sort (10000): 1.48886 sec
